# P3Alpha

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
import os
import csv

from utilities import load_sparse_matrix

### Load dataset

In [ ]:
URM = load_sparse_matrix("urm.npz")
URM_train = load_sparse_matrix("urm_train.npz")
URM_val = load_sparse_matrix("urm_val.npz")
ICM = load_sparse_matrix("icm.npz")

Loaded urm.npz
Loaded urm_train.npz
Loaded urm_val.npz
Loaded icm.npz


In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout

evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[10])

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "shrink": Integer(0, 1000),
    "similarity": Categorical(["cosine"]),
    "normalize": Categorical([True, False]),
}

In [ ]:
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class, evaluator_validation=evaluator_val)

In [ ]:
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)